In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.datasets import mnist
from keras.utils import to_categorical

2024-06-21 12:40:39.735079: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-21 12:40:39.898000: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-21 12:40:39.898092: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-21 12:40:39.924535: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-06-21 12:40:39.975894: I tensorflow/core/platform/cpu_feature_guar

In [2]:
#Loading dataset

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [3]:
#Preprocessing

train_images = train_images.reshape((60000, 784))
train_images = train_images.astype('float32')/255
test_images = test_images.reshape((10000, 784))
test_images = test_images.astype('float32')/255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

In [4]:
from keras import models
from keras.layers import Dense

In [5]:
model = models.Sequential()
model.add(Dense(128, input_dim=784, activation='relu'))
model.add(Dense(10, activation='softmax'))

2024-06-21 12:40:43.127340: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-21 12:40:43.205920: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-06-21 12:40:43.206046: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

In [6]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

print("Training is over")

print("Model survived")
print("Model.summary: ")
print(model.summary())

Training is over
Model survived
Model.summary: 
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 10)                1290      
                                                                 
Total params: 101770 (397.54 KB)
Trainable params: 101770 (397.54 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [7]:
class EBV_Layer(tf.keras.layers.Layer):
    def __init__(self, output_dim, **kwargs):
        self.output_dim = output_dim
        super(EBV_Layer, self).__init__(**kwargs)
    def build(self, input_shape):
        self.kernal = self.add_weight(name='kernal',
                                      shape = (input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(EBV_Layer, self).build(input_shape)
    def call(self, x):
        kernal_norm = tf.norm(self.kernal, axis=0, keepdims=True)
        kernal_norm = tf.maximum(kernal_norm, tf.keras.backend.epsilon())
        kernal_unit_norm = self.kernal/kernal_norm
        output = tf.matmul(x, kernal_unit_norm)
        return output
    
model1 = models.Sequential()
model1.add(Dense(128, input_dim=784, activation='relu'))
model1.add(EBV_Layer(10, input_shape=(128,)))

model1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 128)               100480    
                                                                 
 ebv__layer (EBV_Layer)      (None, 10)                1280      
                                                                 
Total params: 101760 (397.50 KB)
Trainable params: 101760 (397.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
